<a href="https://colab.research.google.com/github/arshbhushan/Resume-Matcher/blob/main/Resume_matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flask import Flask,request,render_template
import os
!pip install docx2txt
import docx2txt
!pip install PyPDF2
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=b72196a8001dcadcd418363b354df7a6b4532efbe7509ce76cdbb5612398e3f1
  Stored in directory: /root/.cache/pip/wheels/0f/0e/7a/3094a4ceefe657bff7e12dd9592a9d5b6487ef4338ace0afa6
Successfully built docx2txt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.6 MB/s eta 0:00:00


In [2]:
app = Flask(__name__)

In [3]:
# set the upload folder in order to save the uploaded files
app.config['UPLOAD_FOLDER'] ='uploads/'

In [4]:
def extract_text_from_pdf(file_path):
  text= ""
  #open file in binary mode:
  with open(file_path,'rb')as file:
    reader= PyPDF2.PdfFileReader(file)

    #extract the text from the pdf
    for page in reader.pages:
      text+= page.extract_text()

  return text

In [5]:
# extract text from docx
def extract_text_from_docx(file_path):
    return docx2txt.process(file_path)

In [6]:
def extract_text_from_txt(file_path):
  with open(file_path, 'r', encoding='utf-8') as file:
    return file.read()

In [7]:
#
def extract_text(file_path):
  if file_path.endswith('.pdf'):
    return extract_text_from_pdf(file_path)

  elif file_path.endswith('.docx'):
    return extract_text_from_docx(file_path)

  elif file_path.endswith('.txt'):
    return extract_text_from_txt(file_path)

  else:
      return "unable to read file, please check the format of the file."

In [11]:
@app.route("/", endpoint='index') # Change endpoint to a unique name like 'index'
def matchresume():
  return render_template('index.html')

In [17]:
# route for handling the resume
def matcher():
  if request.method == 'POST':
    job_description= request.form['job-description']
    resume_files= request.files.getlist('resume')

    resumes= []

    for resume_file in resume_files:
        filename= os.path.join(app.config['UPLOAD_FOLDER'], resume_file.filename)
        resume_file.save(filename)
        resumes.append(extract_text(filename))

    #check for the availability of both the description of the job and resume
    if not resumes or not job_description:
        return render_template('index.html',message="Please upoad resume and enter job description.")

    # vectorise job_description
    vectorizer= TfidfVectorizer().fit_transform([job_description]+resumes)
    vectors= vectorizer.toarray()

    #calculate cosin similarity for matching
    job_vector= vectors[0]
    resume_vectors= vectors[1:]

    similarities= cosine_similarity([job_vector],resume_vectors)

    # get to two results
    top_indices=similarities.argsort()[-2:][::-1]
    top_resume= [resume_files[i].filename for i in top_indices]
    similarity_scores= [round(similarities[i],2) for i in top_indices]

    return render_template('index.html',message=["Top matching resumes:"],top_resume=top_resume,similarity_scores=similarity_scores)

    #render the final template
    return render_template('index.html',message=message,top_resume=top_resume,similarity_scores=similarity_scores)

In [ ]:
# run the app
if __name__ == '__main__':
  # check if the upload folder is created
  if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

  app.run(debug=True)